In [ ]:
while True:
    # Clear the previous plot
    display.clear_output(wait=True)

    dataframe = spark.sql("SELECT * FROM orderQuery").toPandas()
    print(dataframe.to_string())

    time.sleep(3)  


In [2]:
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
from pyspark.sql.functions import count
from IPython import display
import time
from pyspark.sql.functions import from_json, col, sum
from pyspark.sql.types import MapType,StringType,IntegerType
from pyspark.sql.functions import udf, lit

kafka_options = {
    "kafka.bootstrap.servers": "localhost:9092",
    "startingOffsets": "latest", # Start from the latest event when we consume from kafka
    "subscribe": "orders"        # Our topic name
}

spark = SparkSession.builder.appName("IntroToPySpark") \
    .config('spark.jars.packages', 'org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.1') \
    .getOrCreate()

df = spark.readStream.format("kafka").options(**kafka_options).load()

def parse_json_dataframe(json_df):
    def getKey(dictionary, key): 
        return dictionary.get(key)

    udfGetKey = udf(getKey)

    # Deserialize the JSON value from Kafka as a String
    # and convert it to a dict
    return json_df\
        .withColumn("value", from_json(
            col('value').cast("string"),
            MapType(StringType(), StringType())
        )) \
        .withColumn('order_id', udfGetKey(col('value'), lit('order_id'))) \
        .withColumn('amount', udfGetKey(col('value'), lit('amount'))) \
        .withColumn('customer_id', udfGetKey(col('value'), lit('customer_id'))) \
        .select('order_id', 'amount', 'customer_id')

parse_json_dataframe(df) \
    .writeStream \
    .outputMode("append") \
    .format("console") \
    .queryName("ordersQuery")\
    .start() \
    .awaitTermination()

dataframe = spark.sql("SELECT * FROM ordersQuery").toPandas()

while True:
    # Clear the previous plot
    display.clear_output(wait=True)

    dataframe = spark.sql("SELECT * FROM orderQuery").toPandas()
    print(dataframe.to_string())

    time.sleep(3)  


25/10/29 14:59:20 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/m2/7srz5b45113cbcx00jpnxjr40000gp/T/temporary-45a3a2b9-7696-4691-8e23-98efe1aeaa7e. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
25/10/29 14:59:20 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/10/29 14:59:20 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------+------+-----------+
|order_id|amount|customer_id|
+--------+------+-----------+
+--------+------+-----------+

-------------------------------------------
Batch: 1
-------------------------------------------
+--------+------+-----------+
|order_id|amount|customer_id|
+--------+------+-----------+
|      35|    15|          2|
+--------+------+-----------+

-------------------------------------------
Batch: 2
-------------------------------------------
+--------+------+-----------+
|order_id|amount|customer_id|
+--------+------+-----------+
|      36|    25|          7|
+--------+------+-----------+

-------------------------------------------
Batch: 3
-------------------------------------------
+--------+------+-----------+
|order_id|amount|customer_id|
+--------+------+-----------+
|      37|    19|         14|
+--------+------+-----------+

--------------------------------------

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/Users/harrydonnelly/Documents/Proj/stream_processing/stream_processing_venv/lib/python3.11/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/harrydonnelly/Documents/Proj/stream_processing/stream_processing_venv/lib/python3.11/site-packages/py4j/clientserver.py", line 535, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/harrydonnelly/.pyenv/versions/3.11.2/lib/python3.11/socket.py", line 706, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

25/10/29 14:59:37 ERROR Utils: Aborting task
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/Users/harrydonnelly/Documents/Proj/stream_processing/stream_processing_venv/lib/python3.11/site-packages/pyspark/python/lib/pyspark.zip/pyspark/worker.py", line 1968, in main
    split_index = read_int(infile)
                  ^^^^^^^^^^^^^^^^
  File "/Users/harrydonnelly/Documents/Proj/stream_processing/stream_processing_venv/lib/python3.11/site-packages/pyspark/python/lib/pyspark.zip/pyspark/serializers.py", line 595, in read_int
    length = stream.read(4)
             ^^^^^^^^^^^^^^
KeyboardInterrupt

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:581)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:107)
	at org.apache.spark.sql.execution.python.BasePythonUDFRunner$$anon$2.read(PythonUDFRunner.scala:90)
	at org.apache.spark.api.python.BasePytho